# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [35]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tanjung selor,2.8372,117.3642,23.84,97,96,0.55,ID,1723036765
1,1,georgetown,5.4112,100.3354,29.97,68,20,3.09,MY,1723036722
2,2,san patricio,28.0170,-97.5169,28.25,79,0,2.88,US,1723036735
3,3,iqaluit,63.7506,-68.5145,8.85,76,40,1.03,CA,1723036771
4,4,natal,-5.7950,-35.2094,25.36,100,75,4.12,BR,1723036290


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [41]:
%%capture --no-display

# Configure the map plot
mat_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City"
)

# Display the map
mat_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [37]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
ideal_weather_df = city_data_df[
    (city_data_df["Max Temp"] < 27) &
    (city_data_df["Max Temp"] > 21) &
    (city_data_df["Wind Speed"] < 4.5) &
    (city_data_df["Cloudiness"]==0)
]
# Drop any rows with null values
clean_city_df = ideal_weather_df.dropna(how="any")

# Display sample data
clean_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
23,23,saint-pierre,-21.3393,55.4781,22.82,64,0,4.12,RE,1723036793
84,84,karratha,-20.7377,116.8463,22.22,48,0,2.76,AU,1723036865
165,165,port elizabeth,-33.9180,25.5701,26.72,27,0,3.60,ZA,1723036358
221,221,greytown,-29.0642,30.5928,23.32,9,0,2.41,ZA,1723037033
284,284,east end,34.5507,-92.3410,25.93,85,0,0.45,US,1723037106
287,287,hohhot,40.8106,111.6522,24.95,68,0,3.00,CN,1723037110
302,302,albertina,-22.2008,-46.6158,24.89,26,0,3.78,BR,1723037129
303,303,tongliao,43.6125,122.2653,25.50,80,0,3.95,CN,1723037130
333,333,zeerust,-25.5369,26.0751,24.52,20,0,2.93,ZA,1723037181
349,349,belyy yar,53.6039,91.3903,21.94,64,0,2.00,RU,1723037198


### Step 3: Create a new DataFrame called `hotel_df`.

In [38]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = clean_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
23,saint-pierre,RE,-21.3393,55.4781,64,
84,karratha,AU,-20.7377,116.8463,48,
165,port elizabeth,ZA,-33.9180,25.5701,27,
221,greytown,ZA,-29.0642,30.5928,9,
284,east end,US,34.5507,-92.3410,85,
287,hohhot,CN,40.8106,111.6522,68,
302,albertina,BR,-22.2008,-46.6158,26,
303,tongliao,CN,43.6125,122.2653,80,
333,zeerust,ZA,-25.5369,26.0751,20,
349,belyy yar,RU,53.6039,91.3903,64,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [39]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]

    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
saint-pierre - nearest hotel: Tropic Hotel
karratha - nearest hotel: Karratha International Hotel
port elizabeth - nearest hotel: Waterford Hotel
greytown - nearest hotel: Greytown hotel
east end - nearest hotel: No hotel found
hohhot - nearest hotel: Yue Jia Budget Hotel (foreigners approved)
albertina - nearest hotel: Hotel Parque das Primaveras
tongliao - nearest hotel: No hotel found
zeerust - nearest hotel: At Rest Guesthouse
belyy yar - nearest hotel: No hotel found
xilin hot - nearest hotel: Hotel hailibu 120
port saint john's - nearest hotel: Outback Inn
biscarrosse - nearest hotel: Hype Hotel
newman - nearest hotel: No hotel found
correntina - nearest hotel: Pousada Silva
chifeng - nearest hotel: 心连心酒店
hammerfest - nearest hotel: Thon Hotel Hammerfest
darwin - nearest hotel: Mantra Pantanas Darwin


,City,Country,Lat,Lng,Humidity,Hotel Name
23,saint-pierre,RE,-21.3393,55.4781,64,Tropic Hotel
84,karratha,AU,-20.7377,116.8463,48,Karratha International Hotel
165,port elizabeth,ZA,-33.9180,25.5701,27,Waterford Hotel
221,greytown,ZA,-29.0642,30.5928,9,Greytown hotel
284,east end,US,34.5507,-92.3410,85,No hotel found
287,hohhot,CN,40.8106,111.6522,68,Yue Jia Budget Hotel (foreigners approved)
302,albertina,BR,-22.2008,-46.6158,26,Hotel Parque das Primaveras
303,tongliao,CN,43.6125,122.2653,80,No hotel found
333,zeerust,ZA,-25.5369,26.0751,20,At Rest Guesthouse
349,belyy yar,RU,53.6039,91.3903,64,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [42]:
%%capture --no-display

mat_plot2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    hover_cols = ["Country", "Hotel Name"]
)
mat_plot2



:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)